In [2]:
from dividing_data import split_and_save

split_and_save()

Directory validation_data already exists
Directory train_data already exists
validation data saved
test data saved
success! c:


In [19]:
# %load cleaning_data.py
import pandas as pd
import re
import spacy
from langdetect import detect
from collections import Counter
import numpy as np
import string
from unicodedata import category
import sys
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#nlp = spacy.load('en_core_web_sm')


'''def check_languages(df):
    langs = []
    for i in range(len(df['text'])):
        try:
            lang = detect(df['text'][i])
            langs.append(lang)
        except:
            print(i)
    lang_counts = Counter(langs)
    return lang_counts'''


def det_lang(df, text):
    def detect_language(text):
        try:
            lan = detect(text)
        except:
            lan = 'unknown'
        return lan

    df['language'] = df['text'].apply(detect_language)
    return df

        
    

#delete stopwords
def delete_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)
    
   

def clean_text(text, punctuation_chars):
    text = text.lower()

    # remove punctuation
    text = text.translate(str.maketrans('', '', ''.join(punctuation_chars)))
    # remove digits
    text = text.translate(str.maketrans('', '', string.digits))
    # remove all single characters
    pattern = r'(^| ).( |$)'
    text = re.sub(pattern, ' ', text)
    # remove multiple spaces
    text = re.sub(' +', ' ', text)
    return text



def clean_df(df):
    df = df.fillna('')
    punctuation_chars = [chr(i) for i in range(sys.maxunicode)
                         if category(chr(i)).startswith("P")]
    df['text'] = df['text'].map(lambda x: clean_text(x, punctuation_chars))
    df['title'] = df['title'].map(lambda x: clean_text(x, punctuation_chars))
    #
    df['text']=df['text'].apply(delete_stopwords)
    df['title']=df['title'].apply(delete_stopwords)
    #Drop rows where the length of the 4th column is less than 30
    df = df[df['text'].str.len() >= 30]
    df=df.rename(columns={df.columns[0]: 'id'})
    df.dropna(subset=['title','text'], inplace=True)
    languag = det_lang(df,'text')
    #print(languag)
    
    return df

def main():
    # test
    
    df = pd.read_csv('original_data.csv')
    df = clean_df(df)
    df.to_csv('modified_txt.csv', index=False)
    
    #print(df['text'][27])
    #print(df['text'][224])
    #print(df['text'][2])


if __name__ == "__main__":
    main()

df.to_csv('modified_txt.csv', index=False)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bfgam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
df = pd.read_csv('modified_txt.csv')
df.shape


(69045, 4)

In [17]:
df.head(2)

,Unnamed: 0,title,text,Ground Label
0,0,Ann Coulter Make Believes She Has ‘Gay Friend...,"It s hard to believe, but Donald Trump does ha...",fake
1,1,Rating: Moody‘s verbessert Ausblick für Russla...,bankensektor Der russische Staat werde die Ban...,fake


In [18]:
print(df['text'][2])

A member of the House Intelligence Committee is accusing the Obama administration of laundering some $1.7 billion in U.S. taxpayer dollars to Iran through a complicated network that included the New York Federal Reserve and several European banks, according to conversations with sources and new information obtained by the lawmaker and viewed by the Washington Free BeaconNew disclosures made by the Treasury Department to Rep. Mike Pompeo (R., Kan.), a House Intelligence Committee member, show that an initial $400 million cash payment to Iran was wired to the Federal Reserve Bank of New York (FRBNY) and then converted from U.S. dollars into Swiss francs and moved to an account at the Swiss National Bank, according to a copy of communication obtained exclusively by the Free Beacon.Once the money was transferred to the Swiss Bank, the  FRBNY withdrew the funds from its account as Swiss franc banknotes and the U.S. Government physically transported them to Geneva  before personally overseei